# <span style="color:green;">Import </span>

In [1]:
#Scrap 

import requests #Requests allows to send HTTP/1.1 requests

#Utilitaires 
import json
import pandas as pd
import numpy as np
from pandas import json_normalize

from concurrent.futures import ThreadPoolExecutor

#Neo4j

from neo4j import GraphDatabase
from py2neo import Graph
from neomodel import StructuredNode, StringProperty, RelationshipTo, RelationshipFrom, config

# <span style="color:brown;">Scrap Data</span>

In [2]:
occ_url = "https://ec.europa.eu/esco/api/search?language=en&isInScheme=http://data.europa.eu/esco/concept-scheme/occupations&limit=3561"

In [3]:
response = requests.get(occ_url) #Store the request into a new variable

In [4]:
Occupation_source = response.json()

In [5]:
Occupation = Occupation_source['_embedded']

In [6]:
df_Occupation = pd.DataFrame(Occupation["results"]) #Convert to pandas

# <span style="color:brown;">Define List of necessary jobs</span>

In [7]:
DataJobsIndices = df_Occupation['title'].str.contains('op|stewart|learning|artificial|dev|math|intelligence|physics|computer|data|analyst|scientist', na=False)
df_DataJob = df_Occupation[DataJobsIndices]

This code snippet is a filter to include only rows where the 'title' column contains certain keywords related to data and technology.

In [8]:
print(len(df_DataJob))

620


In [9]:
df_DataJob['title']

3                            abrasive blasting operator
4                        absorbent pad machine operator
10                                   accounting analyst
36                               advertising copywriter
61                               agricultural scientist
                             ...                       
3534    Wood processing and papermaking plant operators
3535                    Wood processing plant operators
3538                               wood router operator
3545                  wooden furniture machine operator
3547     Woodworking-machine tool setters and operators
Name: title, Length: 620, dtype: object

In [10]:
#Occupation list from Julien 
Nec_Occ = ['data analyst',
'data scientist',
'computer scientist', 
'chief data officer',
'ICT research consultant',
'data centre operator',
'ICT business analyst',
'data quality specialist',
'data warehouse designer',
'business analyst','data protection officer','database administrator','database designer','database integrator',
'database developer','ICT security manager','knowledge engineer','test engineer','mathematician research engineer',
'bioinformatics scientist']

In [11]:
df_DataJob = df_DataJob[df_DataJob['title'].isin(Nec_Occ)]
df_DataJob = df_DataJob.reset_index()

In [12]:
df_DataJob

,index,className,classId,uri,title,preferredLabel,code,isInScheme,broaderOccupation,_links,isTopConceptInScheme,broaderIscoGroup,broaderConcept
0,337,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/68d973df...,bioinformatics scientist,"{'de': 'Bioinformatiker/Bioinformatikerin', 'n...",2131.3,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C2131],NaN
1,430,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/60082a99...,business analyst,{'de': 'Unternehmensanalyst/Unternehmensanalys...,2421.1,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C2421],NaN
2,547,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/e297ec12...,chief data officer,{'de': 'Leiter der Datenverarbeitung/Leiterin ...,1330.1,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C1330],NaN
3,695,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/e3229e40...,computer scientist,"{'de': 'Informatiker/Informatikerin', 'no': 'd...",2511.1,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C2511],NaN
4,840,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/d3edb8f8...,data analyst,"{'de': 'Datenanalytiker/Datenanalytikerin', 'n...",2511.2,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C2511],NaN
5,841,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/9729c0f3...,data centre operator,{'de': 'Rechenzentrumsbetreiber/Rechenzentrums...,3511.1,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C3511],NaN
6,845,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/7086d0ca...,data quality specialist,{'de': 'Datenqualitätsexperte/Datenqualitätsex...,2519.1,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C2519],NaN
7,846,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/258e46f9...,data scientist,{'de': 'Datenwissenschaftler/Datenwissenschaft...,2511.3,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C2511],NaN
8,847,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/1562c7a3...,data warehouse designer,{'de': 'Data-Warehouse-Entwickler/Data-Warehou...,2521.5,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C2521],NaN
9,848,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/8c57af09...,database administrator,{'de': 'Datenbankadministrator/Datenbankadmini...,2521.1,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C

In [70]:
df_DataJob['title']

0     bioinformatics scientist
1             business analyst
2           chief data officer
3           computer scientist
4                 data analyst
5         data centre operator
6      data quality specialist
7               data scientist
8      data warehouse designer
9       database administrator
10           database designer
11          database developer
12         database integrator
13        ICT business analyst
Name: title, dtype: object

# Data Exploration

In [13]:
df_DataJob['_links'][0]

{'self': {'href': 'https://ec.europa.eu/esco/api/resource/occupation?uri=http://data.europa.eu/esco/occupation/68d973df-bf10-4bf7-9a1b-fbd9f604b9db&language=en',
  'uri': 'http://data.europa.eu/esco/occupation/68d973df-bf10-4bf7-9a1b-fbd9f604b9db',
  'title': 'bioinformatics scientist'}}

In the given JSON object, 'href' and 'uri' are used to represent the same resource, which is an occupation with the title of "bioinformatics scientist". The 'uri' provides a unique identifier for the occupation, while the 'href' provides the URL where the occupation's information can be accessed.

In [14]:
df_DataJob['_links_href'] = df_DataJob['_links'].apply(lambda x: x['self']['href'])

In [15]:
df_DataJob['_links_href']

0     https://ec.europa.eu/esco/api/resource/occupat...
1     https://ec.europa.eu/esco/api/resource/occupat...
2     https://ec.europa.eu/esco/api/resource/occupat...
3     https://ec.europa.eu/esco/api/resource/occupat...
4     https://ec.europa.eu/esco/api/resource/occupat...
5     https://ec.europa.eu/esco/api/resource/occupat...
6     https://ec.europa.eu/esco/api/resource/occupat...
7     https://ec.europa.eu/esco/api/resource/occupat...
8     https://ec.europa.eu/esco/api/resource/occupat...
9     https://ec.europa.eu/esco/api/resource/occupat...
10    https://ec.europa.eu/esco/api/resource/occupat...
11    https://ec.europa.eu/esco/api/resource/occupat...
12    https://ec.europa.eu/esco/api/resource/occupat...
13    https://ec.europa.eu/esco/api/resource/occupat...
Name: _links_href, dtype: object

In [16]:
df_DataJob

,index,className,classId,uri,title,preferredLabel,code,isInScheme,broaderOccupation,_links,isTopConceptInScheme,broaderIscoGroup,broaderConcept,_links_href
0,337,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/68d973df...,bioinformatics scientist,"{'de': 'Bioinformatiker/Bioinformatikerin', 'n...",2131.3,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C2131],NaN,https://ec.europa.eu/esco/api/resource/occupat...
1,430,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/60082a99...,business analyst,{'de': 'Unternehmensanalyst/Unternehmensanalys...,2421.1,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C2421],NaN,https://ec.europa.eu/esco/api/resource/occupat...
2,547,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/e297ec12...,chief data officer,{'de': 'Leiter der Datenverarbeitung/Leiterin ...,1330.1,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C1330],NaN,https://ec.europa.eu/esco/api/resource/occupat...
3,695,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/e3229e40...,computer scientist,"{'de': 'Informatiker/Informatikerin', 'no': 'd...",2511.1,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C2511],NaN,https://ec.europa.eu/esco/api/resource/occupat...
4,840,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/d3edb8f8...,data analyst,"{'de': 'Datenanalytiker/Datenanalytikerin', 'n...",2511.2,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C2511],NaN,https://ec.europa.eu/esco/api/resource/occupat...
5,841,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/9729c0f3...,data centre operator,{'de': 'Rechenzentrumsbetreiber/Rechenzentrums...,3511.1,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C3511],NaN,https://ec.europa.eu/esco/api/resource/occupat...
6,845,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/7086d0ca...,data quality specialist,{'de': 'Datenqualitätsexperte/Datenqualitätsex...,2519.1,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C2519],NaN,https://ec.europa.eu/esco/api/resource/occupat...
7,846,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/258e46f9...,data scientist,{'de': 'Datenwissenschaftler/Datenwissenschaft...,2511.3,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://data.europa.eu/esco/isco/C2511],NaN,https://ec.europa.eu/esco/api/resource/occupat...
8,847,Occupation,http://data.europa.eu/esco/model#Occupation,http://data.europa.eu/esco/occupation/1562c7a3...,data warehouse designer,{'de': 'Data-Warehouse-Entwickler/Data-Warehou...,2521.5,[http://data.europa.eu/esco/concept-scheme/mem...,NaN,{'self': {'href': 'https://ec.europa.eu/esco/a...,[http://data.europa.eu/esco/concept-scheme/mem...,[http://dat

In [17]:
response = requests.get(df_DataJob['_links_href'][0])
response = response.json()

In [18]:
response.keys()

dict_keys(['className', 'classId', 'uri', 'title', 'referenceLanguage', 'preferredLabel', 'alternativeLabel', 'preferredTerm', 'alternativeTerms', 'description', 'status', 'code', 'codes', '_links', '_embedded'])

In [19]:
response['_links']

{'self': {'href': 'https://ec.europa.eu/esco/api/resource/occupation?uri=http://data.europa.eu/esco/occupation/68d973df-bf10-4bf7-9a1b-fbd9f604b9db&language=en',
  'uri': 'http://data.europa.eu/esco/occupation/68d973df-bf10-4bf7-9a1b-fbd9f604b9db',
  'title': 'bioinformatics scientist'},
 'isTopConceptInScheme': [{'href': 'https://ec.europa.eu/esco/api/resource/taxonomy?uri=http://data.europa.eu/esco/concept-scheme/member-occupations&language=en',
   'uri': 'http://data.europa.eu/esco/concept-scheme/member-occupations',
   'title': 'ESCO member occupations'}],
 'isInScheme': [{'href': 'https://ec.europa.eu/esco/api/resource/taxonomy?uri=http://data.europa.eu/esco/concept-scheme/member-occupations&language=en',
   'uri': 'http://data.europa.eu/esco/concept-scheme/member-occupations',
   'title': 'ESCO member occupations'},
  {'href': 'https://ec.europa.eu/esco/api/resource/taxonomy?uri=http://data.europa.eu/esco/concept-scheme/occupations&language=en',
   'uri': 'http://data.europa.eu/e

In [20]:
links = response['_links']

In [71]:
links.keys()

dict_keys(['self', 'isTopConceptInScheme', 'isInScheme', 'regulatedProfessionNote', 'broaderIscoGroup', 'hasEssentialSkill', 'hasOptionalSkill'])

In [21]:
ess_skills = links['hasEssentialSkill']
opt_skills = links['hasOptionalSkill']

In [74]:
ess_skills

[{'href': 'https://ec.europa.eu/esco/api/resource/skill?uri=http://data.europa.eu/esco/skill/6a7792a9-888a-46ad-afc9-2c0d94c8fde6&language=en',
  'uri': 'http://data.europa.eu/esco/skill/6a7792a9-888a-46ad-afc9-2c0d94c8fde6',
  'title': 'create business process models',
  'skillType': 'http://data.europa.eu/esco/skill-type/skill'},
 {'href': 'https://ec.europa.eu/esco/api/resource/skill?uri=http://data.europa.eu/esco/skill/a846cb9e-c152-41e7-bd34-45e7ff941e3b&language=en',
  'uri': 'http://data.europa.eu/esco/skill/a846cb9e-c152-41e7-bd34-45e7ff941e3b',
  'title': 'legal requirements of ICT products',
  'skillType': 'http://data.europa.eu/esco/skill-type/knowledge'},
 {'href': 'https://ec.europa.eu/esco/api/resource/skill?uri=http://data.europa.eu/esco/skill/07e51c60-763a-4335-b4ec-70a15f66d328&language=en',
  'uri': 'http://data.europa.eu/esco/skill/07e51c60-763a-4335-b4ec-70a15f66d328',
  'title': 'propose ICT solutions to business problems',
  'skillType': 'http://data.europa.eu/esc

In [22]:
ess_skills[0]['title'] , ess_skills[0]['skillType']

('manage database', 'http://data.europa.eu/esco/skill-type/skill')

# Skills DF

In [23]:
# Create a list to store the data
data = []

# Loop through the rows of the original DataFrame
for i, row in df_DataJob.iterrows():
    # Get the href value from the _links column
    href = row['_links']['self']['href']
    # Send a request to the API endpoint and get the JSON response
    response = requests.get(href).json()
    # Get the essential and optional skills from the JSON response
    ess_skills = response['_links']['hasEssentialSkill']
    opt_skills = response['_links']['hasOptionalSkill']
    # Get the title from the original DataFrame
    title = row['title']
   # Loop through the essential skills and append them to the "data" list
    for skill in ess_skills:
        skill_type = skill['skillType']
        data.append({'Title': title, 'Href': href, 'Relation': 'essential', 'Skill_title': skill['title'], 'Skill_type': skill_type})
# Loop through the optional skills and append them to the "data" list
    for skill in opt_skills:
        skill_type = skill['skillType']
        data.append({'Title': title, 'Href': href, 'Relation': 'optional', 'Skill_title': skill['title'], 'Skill_type': skill_type})

# Create a new DataFrame from the data list
df_skills = pd.DataFrame(data)

## Clean df

In [24]:
# Get the last part of the string after the last "/"
df_skills['Skill_type'] = df_skills['Skill_type'].apply(lambda x: x.split("/")[-1])
df_skills

,Title,Href,Relation,Skill_title,Skill_type
0,bioinformatics scientist,https://ec.europa.eu/esco/api/resource/occupat...,essential,apply scientific methods,skill
1,bioinformatics scientist,https://ec.europa.eu/esco/api/resource/occupat...,essential,present reports,skill
2,bioinformatics scientist,https://ec.europa.eu/esco/api/resource/occupat...,essential,use databases,skill
3,bioinformatics scientist,https://ec.europa.eu/esco/api/resource/occupat...,essential,contact scientists,skill
4,bioinformatics scientist,https://ec.europa.eu/esco/api/resource/occupat...,essential,gather data,skill
...,...,...,...,...,...
771,ICT business analyst,https://ec.europa.eu/esco/api/resource/occupat...,optional,visual presentation techniques,knowledge
772,ICT business analyst,https://ec.europa.eu/esco/api/resource/occupat...,optional,provide user documentation,skill
773,ICT business analyst,https://ec.europa.eu/esco/api/resource/occupat...,optional,information architecture,knowledge
774,ICT business analyst,https://ec.europa.eu/esco/api/resource/occupat...,optional,cloud technologies,knowledge


# Occupation DF
for main node

In [25]:
response.keys()

dict_keys(['className', 'classId', 'uri', 'title', 'referenceLanguage', 'preferredLabel', 'alternativeLabel', 'preferredTerm', 'alternativeTerms', 'description', 'scopeNote', 'status', 'code', 'codes', '_links', '_embedded'])

In [26]:
response['title'], response['preferredLabel']['fr'], response['alternativeLabel']['en'], response['description']['en']


('ICT business analyst',
 'analyste d’étude en TIC',
 ['ICT business analysts',
  'business process specialist',
  'IT business analyst'],
 {'literal': "ICT business analysts are in charge of analysing and designing an organisation's processes and systems, assessing the business model and its integration with technology. They also identify change needs, assess the impact of the change, capture and document requirements and then ensure that these requirements are delivered whilst supporting the business through the implementation process.",
  'mimetype': 'plain/text'})

In [27]:
response['alternativeLabel']['en']

['ICT business analysts', 'business process specialist', 'IT business analyst']

In [28]:
# Create an empty list to store the data
data = []

# Iterate over the _links_href column and extract the required data
for href in df_DataJob['_links_href']:
    # Send a request to the API endpoint and get the JSON response
    response = requests.get(href).json()
    # Extract the required data from the response
    title = response['title']
    pref_label = response['preferredLabel']['fr']
    alt_label = ", ".join([label['value'] if isinstance(label, dict) else label for label in response['alternativeLabel']['en']])
    desc = response['description']['en']
    # Append the data to the "data" list
    data.append({'Title': title, 'PreferredlabelinFR': pref_label, 'AltLabel': alt_label, 'Description': desc})

# Create a new DataFrame from the data list
df_new = pd.DataFrame(data, columns=['Title', 'PreferredlabelinFR', 'AltLabel', 'Description'])



In [29]:
df_new['Description'][0]

{'literal': 'Bioinformatics scientists analyse biological processes using computer programmes. They maintain or construct databases containing biological information. Bioinformatics scientists gather and analyse biological data and may also assist scientists in various fields, including in biotechnology and pharmaceutics. They perform scientific research and statistical analyses, and report on their findings. Bioinformatics scientists may also collect DNA samples, discover data patterns and conduct genetic research.',
 'mimetype': 'plain/text'}

In [30]:
# clean the 'Description' column
for index, row in df_new.iterrows():
    description = row['Description']['literal']
    cleaned_description = description.replace('\'', '').replace('{', '').replace('}', '').replace('literal:', '').replace('mimetype:plain/text,', '')
    df_new.at[index, 'Description'] = cleaned_description

df_new['Description'][0]

'Bioinformatics scientists analyse biological processes using computer programmes. They maintain or construct databases containing biological information. Bioinformatics scientists gather and analyse biological data and may also assist scientists in various fields, including in biotechnology and pharmaceutics. They perform scientific research and statistical analyses, and report on their findings. Bioinformatics scientists may also collect DNA samples, discover data patterns and conduct genetic research.'

In [31]:
df_new['Description'][2]

'Chief data officers manage companies enterprise-wide data administration and data mining functions. They ensure data are used as a strategic business asset at the executive level and implement and support a more collaborative and aligned information management infrastructure for the benefit of the organisation at large.'

In [32]:
df_occupation = df_new

# Final DF

In [33]:
df_skills

,Title,Href,Relation,Skill_title,Skill_type
0,bioinformatics scientist,https://ec.europa.eu/esco/api/resource/occupat...,essential,apply scientific methods,skill
1,bioinformatics scientist,https://ec.europa.eu/esco/api/resource/occupat...,essential,present reports,skill
2,bioinformatics scientist,https://ec.europa.eu/esco/api/resource/occupat...,essential,use databases,skill
3,bioinformatics scientist,https://ec.europa.eu/esco/api/resource/occupat...,essential,contact scientists,skill
4,bioinformatics scientist,https://ec.europa.eu/esco/api/resource/occupat...,essential,gather data,skill
...,...,...,...,...,...
771,ICT business analyst,https://ec.europa.eu/esco/api/resource/occupat...,optional,visual presentation techniques,knowledge
772,ICT business analyst,https://ec.europa.eu/esco/api/resource/occupat...,optional,provide user documentation,skill
773,ICT business analyst,https://ec.europa.eu/esco/api/resource/occupat...,optional,information architecture,knowledge
774,ICT business analyst,https://ec.europa.eu/esco/api/resource/occupat...,optional,cloud technologies,knowledge


In [34]:
df_occupation

,Title,PreferredlabelinFR,AltLabel,Description
0,bioinformatics scientist,bioinformaticien/bioinformaticienne,"environmental scientist, bioinformatics analys...",Bioinformatics scientists analyse biological p...
1,business analyst,analyste de gestion,"standardisation expert, business strategy anal...",\nBusiness analysts research and understand th...
2,chief data officer,directeur des données/directrice des données,"chief data officers, chief analytics officer, CDO",Chief data officers manage companies enterpris...
3,computer scientist,informaticien/informaticienne,"ICT scientist, computer scientist, IT research...",Computer scientists conduct research in comput...
4,data analyst,analyste de données,"data warehousing analyst, data analysts, data ...","Data analysts import, inspect, clean, transfor..."
5,data centre operator,opérateur de centre de données/opératrice de c...,"data center administrator, data centre operato...",Data centre operators maintain computer operat...
6,data quality specialist,analyste qualité des données,"data quality expert, data integrity officer, d...",Data quality specialists review organisations ...
7,data scientist,scientifique des données,"data scientists, data engineer, research data ...",Data scientists find and interpret rich data s...
8,data warehouse designer,concepteur d’entrepôt de données/conceptrice d...,"data warehouse architect, data warehouse devel...",Data warehouse designers are responsible for p...
9,database administrator,administrateur de base de données/administratr...,"database manager, DBA, database administrators...","Database administrators test, implement and ad..."


# <span style="color:blue;">Neo4j</span>

In [35]:
class Neo4jConnection: #Init class
    '''   
    Python class called Neo4jConnection that can be used to connect to a Neo4j database 
    using the official Python driver GraphDatabase.driver.
    The __init__ method initializes the class with the database URI, username, and password. 
    The close method closes the database driver if it is open. 
    The query method executes a Cypher query with optional parameters and returns the query response as a list.
 
    '''
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response


In [36]:
#Connect to DB
conn = Neo4jConnection(uri="bolt://18.213.115.214:7687", 
                       user="neo4j",              
                       pwd="fatigues-standing-credibility")

# <span style="color:blue;">Clean db</span>

In [ ]:
#Don't run (only to reset db)
clean = conn.query("MATCH (n) DETACH DELETE n ")
clean

# <span style="color:blue;">Create Label</span>

In [ ]:
conn.query('CREATE CONSTRAINT title IF NOT EXISTS FOR (t:Title)     REQUIRE t.title IS UNIQUE')
conn.query('CREATE CONSTRAINT skills IF NOT EXISTS FOR (s:Skills)     REQUIRE s.skills IS UNIQUE')
conn.query('CREATE CONSTRAINT knowledge IF NOT EXISTS FOR (k:Knowledge)     REQUIRE k.knowledge IS UNIQUE')

## <span style="color:blue;">Title</span>

In [37]:
def add_title(title): 
    """
    This is a Python function called add_title that takes in a pandas DataFrame 
    and executes a Cypher query on a Neo4j database connected by a conn object 
    
    The Cypher query in this function is designed to create a Title node in the database.
    With properties title, description, and alternativeLabel. 
    
    """
    
    query = '''
        UNWIND $rows AS row
        MERGE (t:Title {title: row.Title})
        ON CREATE SET t.description = row.Description, t.alternativeLabel = row.AltLabel,
        t.preferredLabelFR = row.PreferredlabelinFR
        RETURN count(*) as total
        '''

    #The ON CREATE SET clause sets the description and alternativeLabel properties for the newly created Title node.
    return conn.query(query, parameters={'rows': title.to_dict('records')})

In [65]:
add_title(df_occupation)

[<Record total=14>]

## <span style="color:blue;">Skills/Knowledge</span>

In [69]:
def create_skill_nodes(conn, df_skills):
    """This is a Python function called create_skill_nodes that takes in a pandas DataFrame 
    and executes  Cypher queries on a Neo4j database connected by a conn object 
    
    The Cypher queries in this function are designed to create a Skills and Knowledge nodes in the database.
    With properties href and "essential" or "optional" relation to the nodes t:Title. """
    
    for _, row in df_skills.iterrows():
        title_name = row['Title']
        skill_title = row['Skill_title']
        skill_type = row['Skill_type']
        relation = row['Relation']
        href = row['Href']

        # Determine the node label based on the skill type
        if skill_type == 'skill':
            node_label = 'Skills'
        elif skill_type == 'knowledge':
            node_label = 'Knowledge'
        else:
            raise ValueError(f"Invalid skill type: {skill_type}")

        # First, check if the skill already exists in the database
        existing_skills = conn.query(f"MATCH (s:{node_label} {{skills: $skill_title}}) RETURN s", {'skill_title': skill_title})
        if existing_skills:
            skill_node = existing_skills[0]['s']
        else:
            # If the skill doesn't exist, create a new node
            skill_node = conn.query(f"CREATE (s:{node_label} {{skills: $skill_title, Href: $href}}) RETURN s", {'skill_title': skill_title, 'href': href})[0]['s']

        # Get the Title node
        title_node = conn.query("MATCH (t:Title {title: $title_name}) RETURN t", {'title_name': title_name})[0]['t']

        # Create the relationship between the Title and the skill node
        conn.query("MATCH (t:Title {title: $title_name}), (s) WHERE ID(s) = $skill_id CREATE (t)-[r:" + relation + "]->(s)", {'title_name': title_name, 'skill_id': skill_node.id})


In [68]:
#Take time
create_skill_nodes(conn, df_skills)

StopIteration: 